# Set up

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from ortools.linear_solver import pywraplp
import numpy as np
import os


<IPython.core.display.Javascript object>

In [3]:
def main():
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP','SCIP')

    infinity = solver.infinity()
    # x and y are integer non-negative variables.
    x = solver.IntVar(0, infinity, 'x')
    y = solver.IntVar(0, infinity, 'y')

    print('Number of variables =', solver.NumVariables())

    # x + 7 * y <= 17.5.
    solver.Add(x + 7 * y <= 17.5)

    # x <= 3.5.
    solver.Add(x <= 3.5)

    print('Number of constraints =', solver.NumConstraints())

    # Maximize x + 10 * y.
    solver.Maximize(x + 10 * y)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print('x =', x.solution_value())
        print('y =', y.solution_value())
    else:
        print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())




<IPython.core.display.Javascript object>

In [4]:
main()

Number of variables = 2
Number of constraints = 2
Solution:
Objective value = 23.0
x = 3.0
y = 2.0

Advanced usage:
Problem solved in 11.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


<IPython.core.display.Javascript object>

# Knapsack

In [24]:
path = '../2 knapsack/data'
files = os.listdir(path)


<IPython.core.display.Javascript object>

In [25]:
for i, f in enumerate(files):
    print(i, f)

0 ks_10000_0
1 ks_1000_0
2 ks_100_0
3 ks_100_1
4 ks_100_2
5 ks_106_0
6 ks_19_0
7 ks_200_0
8 ks_200_1
9 ks_300_0
10 ks_30_0
11 ks_400_0
12 ks_40_0
13 ks_45_0
14 ks_4_0
15 ks_500_0
16 ks_50_0
17 ks_50_1
18 ks_60_0
19 ks_82_0
20 ks_lecture_dp_1
21 ks_lecture_dp_2


<IPython.core.display.Javascript object>

In [26]:
def parse_data(data):
    lines = data.split("\n")
    n, cap = lines[0].split()
    data = [line.split() for line in lines if len(line) > 0]
    n, cap = data.pop(0)
    return np.int(n), np.float(cap), np.array(data, dtype=float)

<IPython.core.display.Javascript object>

In [40]:
file = files[0]
with open(os.path.join(path, file)) as fp:
    data = fp.read()

n, K, data = parse_data(data)
values = data[:, 0]
weights = data[:, 1]

<IPython.core.display.Javascript object>

In [38]:
def ks_mip(weights, values, capacity):
    assert len(weights) == len(values), "Weights and Values must be of the same size."

    # Create solver
    solver = pywraplp.Solver.CreateSolver("SCIP", "SCIP")

    # Define Variables
    n = len(weights)
    choices = [solver.BoolVar(f"x_{i}") for i in range(n)]

    # Constraints
    total_weight = sum([weights[i] * choices[i] for i in range(n)])
    solver.Add(total_weight <= capacity)

    # Objective
    total_value = sum([values[i] * choices[i] for i in range(n)])
    solver.Maximize(total_value)

    # Solve
    status = solver.Solve()

    # Solution
    if status == pywraplp.Solver.OPTIMAL:
        solution = [choices[i].solution_value() for i in range(n)]
        total_weight = sum([weights[i] * solution[i] for i in range(n)])
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value()}")
        print(f"Weight = {total_weight}/{capacity}")
        print(solution)
    #         print('x =', x.solution_value())
    #         print('y =', y.solution_value())
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())

<IPython.core.display.Javascript object>

In [ ]:
ks_mip(weights, values, capacity=K)

# Graph Coloring

In [3]:
path = "../3 coloring/data"
files = os.listdir(path)
[f"{i}. {f}" for i, f in enumerate(files)]

['0. gc_1000_1',
 '1. gc_1000_3',
 '2. gc_1000_5',
 '3. gc_1000_7',
 '4. gc_1000_9',
 '5. gc_100_1',
 '6. gc_100_3',
 '7. gc_100_5',
 '8. gc_100_7',
 '9. gc_100_9',
 '10. gc_20_1',
 '11. gc_20_3',
 '12. gc_20_5',
 '13. gc_20_7',
 '14. gc_20_9',
 '15. gc_250_1',
 '16. gc_250_3',
 '17. gc_250_5',
 '18. gc_250_7',
 '19. gc_250_9',
 '20. gc_4_1',
 '21. gc_500_1',
 '22. gc_500_3',
 '23. gc_500_5',
 '24. gc_500_7',
 '25. gc_500_9',
 '26. gc_50_1',
 '27. gc_50_3',
 '28. gc_50_5',
 '29. gc_50_7',
 '30. gc_50_9',
 '31. gc_70_1',
 '32. gc_70_3',
 '33. gc_70_5',
 '34. gc_70_7',
 '35. gc_70_9']

<IPython.core.display.Javascript object>

In [15]:
idx = 10
print(files[idx])
with open(os.path.join(path, files[idx]), "r") as fp:
    input_data = fp.read()
print(input_data[:15])

gc_20_1
20 23
0 16
1 2



<IPython.core.display.Javascript object>

In [16]:
lines = input_data.split("\n")

first_line = lines[0].split()
node_count = int(first_line[0])
edge_count = int(first_line[1])

edges = []
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))

print(node_count, edge_count)

20 23


<IPython.core.display.Javascript object>

In [17]:
edge_array = np.array(edges)
orders = [np.sum(edge_array == i) for i in range(node_count)]
highest_order = int(np.argmax(orders))
print(highest_order, orders[highest_order])

17 5


<IPython.core.display.Javascript object>

In [18]:
max_color = highest_order + 1

<IPython.core.display.Javascript object>

In [19]:
def gc_mip(n_points, edges, max_color):
    n_colors = max_color

    # Create solver
    solver = pywraplp.Solver.CreateSolver("SCIP", "SCIP")

    # Define Variables
    colors = [
        [solver.BoolVar(f"x_{i}_{c}") for c in range(n_colors)] for i in range(n_points)
    ]

    # Constraints
    # Connected edges
    for i, j in edges:
        for c in range(n_colors):
            solver.Add(colors[i][c] + colors[j][c] <= 1)

    # Only one color
    for i in range(n_points):
        solver.Add(sum(colors[i]) <= 1)
        solver.Add(sum(colors[i]) >= 1)

    #     total_weight = sum([weights[i] * choices[i] for i in range(n)])
    #     solver.Add(total_weight <= capacity)

    # Objective
    obj = 0
    for i in range(n_points):
        obj += sum([colors[i][c] * c for c in range(n_colors)])
    solver.Minimize(obj)

    # Solve
    status = solver.Solve()

    # Solution
    if status == pywraplp.Solver.OPTIMAL:
        #         solution = [choices[i].solution_value() for i in range(n)]
        solution = [
            [colors[i][c].solution_value() for c in range(n_colors)]
            for i in range(n_points)
        ]
        for i in range(n_points):
            print(solution[i])
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value()}")

    #         print(solution)
    #         print('x =', x.solution_value())
    #         print('y =', y.solution_value())
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())

<IPython.core.display.Javascript object>

In [20]:
gc_mip(node_count, edges, max_color)

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

<IPython.core.display.Javascript object>

In [13]:
solver = pywraplp.Solver.CreateSolver("SCIP", "SCIP")

<IPython.core.display.Javascript object>

In [ ]:
solver.Minimize